## Assignment #1
Module code: ID5059

Module: Knowledge Discovery and Datamining

In [ ]:
# Import required 
import sys
!{sys.executable} -m pip install numpy pandas matplotlib scikit-learn | grep -v 'already satisfied'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Get the data by reading the dataset from a saved locally
# Tested on larger datasets, but submitted on the smaller one for data size considerations
used_cars_data = pd.read_csv("data/1_small/used_cars_data_small_0.csv")

In [ ]:
# Get some basic understanding of the data and its structure
print(used_cars_data.head(10))
print(used_cars_data.info())
print(used_cars_data.describe(include="all"))

### Cleaning the Data

In obtaining some understanding of what the data looks like, it becomes evident that data cleaning is necessary for useful results to be yielded later in the machine learning process. There are a lot of variables, data types, some of which are not useful in predicting the price, missing values and different formatting. As such, the data is cleaned before it can be used for computations. For this assignment, I have chosen to keep a separate variable for the used cars for the data exploration and data cleaning, so that I can go back and look at the results later on.

In [ ]:
# Some of the columns in the dataset had all NaN values. These will in no way be helpful to future computations, and are therefore removed
# We are then left with 63 columns instad of 66, when using the used_cars_data_small_0.csv dataset
used_cars_data.dropna(axis=1, inplace=True, how="all")

# However, we might want to run this code on other datasets
# Therefore, we add contingency code to drop the tables anyways, should they still exist
columns_to_drop = ["combine_fuel_economy", "is_certified", "vehicle_damage_category"]
for col in columns_to_drop:
    if col in used_cars_data.columns:
        used_cars_data.drop(col, axis=1, inplace=True, how="all")

# Attributes that likely have no significance on calculating price are also dropped
used_cars_data = used_cars_data.drop(labels=["is_cpo", "is_oemcpo", "sp_id", "exterior_color", "listing_id", "description", "main_picture_url", "vin", "bed_height", "bed_length", "cabin"], axis=1)

# Some of the values of attributes are indicated as missing by the "--" notation
used_cars_data.replace("--", np.nan, inplace=True)

In [ ]:
# A second look at the data frame reveales that some values will be complicated to work with
# Even if processed through mapping of categories, some attributes have too many unique values to be worth the efforts, and are not useful in predicting the price anyways
# Such attributes are therefore dropped too
used_cars_data = used_cars_data.drop(labels=["bed", "fleet", "frame_damaged","franchise_make", "isCab", "has_accidents", "owner_count", "salvage", "theft_title", "highway_fuel_economy", "transmission_display", "torque"], axis=1)

# Some attributes are not useful in the light of other attributes, such as "model_name," since "make_name" is sufficient in this context
used_cars_data = used_cars_data.drop(labels=["engine_cylinders", "engine_displacement", "listed_date", "model_name", "interior_color", "major_options", "trimId", "trim_name", "dealer_zip", "sp_name"], axis=1)

In [ ]:
# Convert some of the attributes to a more usable format
used_cars_data["height"] = used_cars_data["height"].str.replace(r' in', '')
used_cars_data["height"] = pd.to_numeric(used_cars_data["height"])

# Then for the width...
used_cars_data["width"] = used_cars_data["width"].str.replace(r' in', '')
used_cars_data["width"] = pd.to_numeric(used_cars_data["width"])

# ... and wheelbase
used_cars_data["wheelbase"] = used_cars_data["wheelbase"].str.replace(r' in', '')
used_cars_data["wheelbase"] = pd.to_numeric(used_cars_data["wheelbase"])

# Next, the front_legroom attribute is converted
used_cars_data["front_legroom"] = used_cars_data["front_legroom"].str.replace(r' in', '')
used_cars_data["front_legroom"] = pd.to_numeric(used_cars_data["front_legroom"])

# Convert the back_legroom attribute
used_cars_data["back_legroom"] = used_cars_data["back_legroom"].str.replace(r' in', '')
used_cars_data["back_legroom"] = pd.to_numeric(used_cars_data["back_legroom"])

# Same process for the fuel_tank_volume attribue
used_cars_data["fuel_tank_volume"] = used_cars_data["fuel_tank_volume"].str.replace(r' gal', '')
used_cars_data["fuel_tank_volume"] = pd.to_numeric(used_cars_data["fuel_tank_volume"])

# ... and power, removing "hp" and everything after it
used_cars_data["power"] = used_cars_data["power"].astype("string")
used_cars_data["power"] = used_cars_data["power"].str[:-15]
used_cars_data["power"] = pd.to_numeric(used_cars_data["power"])


In [ ]:
# Make sure everything looks as intended after data cleaning
used_cars_data.head(10)
used_cars_data.info()

In [ ]:
# Histograms can also be useful in understanding the data for numerical attributes
used_cars_data.hist(bins=30, figsize=(15,15))
plt.show()

### Exploring the Data #1

In exploring the data structure, some interesting and potentially relevant obersvations emerge. While it is important not to look to closely at the data before processing it in machine learning, accounting for how the human brain is very good at recognising patterns and could therefore easily be biased by findings if too much time is spent exploring the data structure, it is still important to extract the key point which will affect how we proceed. Interesting observations from exploring the data structure include:
* Certain attributes have no null values, such as vin, price, city, make and year.
* There are several attribute types, where many have the type string. This must be considere4d before completing future calculations on the data set.
* Some attributes have a tail-heavy distribution, where most values are located on one side of the x-axis. Some examples of tail-heavy attributes are daysonmarket and year.

### Creating a Test Set

Creating a test set and a training set is essential to later on see how well the models perform on test data after having been trained on a portion of the data. The test set is therefore set aside early on, and not considered again until we are ready to test the models. It is common to use a 80-20 set, which we will do in this case too. However, there is still some more work to do before we can create the test and training set, so let us creste the test set, work some more with the data, and then set up a stratified test set.

In [ ]:
# Set the final version of the clean dataset to be equal to a new variable, called "cars"
cars = used_cars_data
cars = cars.fillna(cars.median())

It is important not to introduce sampling bias, when working with stratified sampling. This would be is data that is generally representative of the data no longer is representing the appropriate instances. In this scenario, we would want the attributes that might be important predictors of the price to stay representative of their respective categories. As such, to make sure certain categories maintain a matching distribution, we Pandas cut() method for the attribute milage.

In [ ]:
# Begin the process of creating a test set and a training set
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(cars, test_size=0.2, random_state=42)

# Confirm the results look correct
len(test_set) / len(cars)

In [ ]:
# Accounting for the representation of mileage, by creating categories for the mileage attribute
cars["mileage_category"] = pd.cut(cars["mileage"], bins=[0.0, 50000.0, 100000.0, 150000.0, 200000.0, np.inf], labels=[1,2,3,4,5])

In [ ]:
# Check if the new bins we created contain enough values
cars["mileage_category"].value_counts()

In [ ]:
# Check if the newly created categories are created as intended
cars["mileage_category"].hist()
plt.show()

In [ ]:
# The mileage attribute has been added with the data type "category," and must therefore be converted to a numerical data type
print(cars.dtypes)

In [ ]:
# Convert the datatype of the cars["mileage_category"] from category to integer
cars["mileage_category"] = cars["mileage_category"].astype('float64')
print(cars.dtypes)

In [ ]:
# Ensure there are no NaN values for our numeric values, in order to be able to do the StratifiedShuffleSplit
# Check to see if there are any, and then we replace these with the median value, as not to affect the results
print(cars.isna().sum())
cars = cars.fillna(cars.median())

In [ ]:
# Use the algorithm for stratispied splitting from Scikit-learn to treat the newly created category
from sklearn.model_selection import StratifiedShuffleSplit
shuffled_data = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

[(train_index, test_index)] = shuffled_data.split(cars, cars["mileage_category"])
stratified_train_set = cars.loc[train_index]
stratified_test_set = cars.loc[test_index]

In [ ]:
# Now we see how we did for the test set...
stratified_test_set["mileage_category"].value_counts() / len(stratified_test_set)

In [ ]:
# ... and for the training set
stratified_train_set["mileage_category"].value_counts() / len(stratified_train_set)

Considering how the resulting values from testing the value counts in the two data sets are relatively similar, and we can therefore feel comfortable about the quality of the split. As such, we can move forward with process, and the next step would be to remove the newly created column, in addition to the original data set.

In [ ]:
# Get rid of the new column and the original dataset
del(cars)
stratified_train_set.drop(columns="mileage_category", inplace=True)
stratified_test_set.drop("mileage_category", axis=1, inplace=True)

stratified_train_set.head()

Having done so, it is time to store the test set, so that we do not look at it again accidentally or on purpose. If we need to or want to look back at the data, the only information available will be the training set.

In [ ]:
# Create a copy of the training set
cars = stratified_train_set.copy()

### Exploring the Data #2

Having created a test set and put it aside, we can now take a proper look at the data, in order to get a better understanding of what attributes could possibly be correlated with the price. Exploring correlations will be a good place to start, to see what attributes look like they are most likely correlated with the price.

In [ ]:
# Use the Standard Correlation Coefficient to start looking for correlation, to determine what attributes should be more closely investigated
corr_matrix = cars.corr()

# Explore the correlation for price, ordered by their value in descending order
corr_matrix["price"].sort_values(ascending=False)

When interpreting the results from the correlation, the results can range from 1 to -1, where results close to 1 indicates a strong positive correlation, and results close to -1 indicates a strong negative correlation.

In [ ]:
# Check for correlation using the scatter_matrix() function, but this is primarily doable on smaller datasets
# Create one for the numeric attributes we believe to be most closely correlated with the price
from pandas.plotting import scatter_matrix

attributes = ["price", "year", "horsepower", "power", "mileage", "longitude"]
pd.plotting.scatter_matrix(cars[attributes], figsize=(15, 15))
plt.show()

In [ ]:
# Mileage and year looks to be the most interesting attributes to explore further, based on the scatter matrix
cars_correlations = cars.corr(numeric_only=True)
cars_correlations

# We are primarily interested in correlations to the price, and will therefore focus on that
cars_correlations["price"].sort_values(ascending=False)

In [ ]:
# Year looks to be closely related to price. We therefore take a closer look at this scatter plot
plt.scatter(cars['price'], cars['year'])
plt.title('Price vs. Year')
plt.xlabel('Price')
plt.ylabel('Year')
plt.show()

After having explored the data, we see that year has a relatively strong correlation, while mileage has a relatively strong negative correlation. These are both helpful findings, as they are good indicators that these two attributes might have a significance in predicting the price of the used cars.

### Prepare the data

Now, after having investigated the data, we need to prepare it. In doing so, we must separate the labels from the features, and drop the column that is for "price" itself. In preparing the data, we must also ensure that there are non NaN or non-null values.

In [ ]:
# Separate labels from the features
cars_labels = cars["price"].copy()
cars_labels.head()

In [ ]:
# Drop the column for the "price"
cars = cars.drop(columns="price")
cars.head()

In [ ]:
# Check if any of the data values are missing
cars.info()

In [ ]:
# There are some missing values, as not all attributes have all non-null values, and these are replaced with the median value
fill_values = cars.select_dtypes(include=['int', 'float']).columns
for col in fill_values:
    median_val = cars[col].median()
    cars[col].fillna(median_val, inplace=True)

# Check the data values again
cars.info()

In [ ]:
# Moving on to the boolean values and object values
# As the imputer only wants numeric values, we must first drop these, then the array must be transformed into a data frame
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")
drop_attributes = cars.select_dtypes(include=['bool', 'object']).columns
cars_numerical = cars.drop(drop_attributes, axis=1)

raw_output = imputer.fit_transform(cars_numerical)
cars_imputed = pd.DataFrame(raw_output, columns=cars_numerical.columns, index=cars_numerical.index)

In [ ]:
# To do a quality check, we check if any of the numerical attributes still have null values
null_count = cars_numerical.isnull().sum()
print(null_count)

In [ ]:
# However, we might still want to take a look at some of the boolean and object attributes
# These must therefore be converted to a more useful format
cars_categorical = cars[["franchise_dealer", "is_new", "body_type", "city", "engine_type", "fuel_type", "length", "listing_color", "make_name", "maximum_seating", "transmission", "wheel_system", "wheel_system_display"]]

cars_categorical

In encoding these attributes, there are several ways this could be done. One way would be to encode the text into numbers with the OrdinalEncoder() method, but this might attribute too much weights to the numbers, which will skew the results. As such, we will use the one-hot encoding to replace the bollean and object attributes' true value with zeros and ones, indicating whether this attribute is present or not.

In [ ]:
# As we cannot do calculations on values that are not numeric, we use the OneHodEncoder() method
from sklearn.preprocessing import OneHotEncoder
categorical_encoder = OneHotEncoder()

cars_categorical_encoded = categorical_encoder.fit_transform(cars_categorical)
cars_categorical_encoded

# Convert the result to an array, as it originally is returned as a sparse matrix
cars_categorical_encoded.toarray()

Next, we might want to scale our numerical features, as machine learning algorithms tend to work best when there is a similar scale to the numbers. We therefore modify the numerical features (Géron, 2019). The code is based on examples covered and learned in ID5059 lectures, where names of variables have been replaced with attributes more appropriate for the context of the US Used Cars dataset.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):

    # This example shows how to give the constructor an option
    # The transformer will always add the rooms_per_household and population_per_household features
    # We use an input argument to control whether to add (the default choice) or not the bedrooms_per_room feature
    def __init__(self, add_mileage_per_year = True): 
        self.add_mileage_per_year = add_mileage_per_year

    # This estimator only transforms, so the fit() method doesn't do anything
    def fit(self, X, y=None):
        return self

    # Our transformation
    def transform(self, X):

        # The input dataset X is treated as a numpy array, so we will identify features by column number
        years_index = 3
        mileage_index = 4
        make_name_index = 5
        model_name_index = 6

        # Compute two new features
        years_per_model_name = X[:, years_index] / X[:, model_name_index]
        make_name_per_model_name = X[:,  make_name_index] / X[:, model_name_index]

        # Make a third new feature and then returns as output the input with all three new features added as new columns
        if self.add_mileage_per_year:
            mileage_per_year = X[:, mileage_index] / X[:, years_index]
            return np.c_[X, years_per_model_name, make_name_per_model_name, mileage_per_year]

        # Only returns as output the input with only the two new features added as new columns
        else:
            return np.c_[X, years_per_model_name, make_name_per_model_name]

### Transformation Pipelines

One example of a pipeline we could make for the dataset, is a preparation pipeline for data of numerical value, which fills inn null entries with the median value, adds new features better correlating to the labels and standardise everything. Such a pipeline is created below.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

numerical_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attributes_adder', CombinedAttributesAdder()),
        ('scaler', StandardScaler()),
    ])

In [ ]:
# Use the pipeline by using the fit_transform method
cars_numerical_transformed = numerical_pipeline.fit_transform(cars_numerical)

# Check to see if everything looks right
pd.DataFrame(cars_numerical_transformed).info()

In [ ]:
# As the previous pipeline only can be used on numbers, we can create a new one for the categorical attributes
# We can use scikit-learn's ColumnTransformer() method, built for combining such pipelines
from sklearn.compose import ColumnTransformer

# Since is_new was the only attribute that was not a float or an integer, but that still was of real interest in terms of correlation, we only bring this in the pipeline
numerical_attributes = list(cars_numerical)
categorical_attributes = ["is_new"]

full_pipeline = ColumnTransformer([
    ("numerical", numerical_pipeline, numerical_attributes),
    ("categorical", OneHotEncoder(), categorical_attributes),
])

In [ ]:
# See what the whole pipeline looks like.
cars_prepared = full_pipeline.fit_transform(cars)

# Have a look at what the results look like...
pd.DataFrame(cars_prepared).info()

# ... and what the first few rows will display
pd.DataFrame(cars_prepared).head()

In [ ]:
# Now have more columns, which is likely related to the is_new attribute
# Check to confirm this, and as suspected, it is a boolean value with only two possible options
cars["is_new"].value_counts()

### Moving on to Machine Learning Models
Finally, after a lot of data exploration, cleaning and preparation, we can look more closely at some Machine Learning models, and how they interpret and perform on the dataset. We start off with Linear Regression, to generate some predictions for the cars test data.

In [ ]:
# Fit a least squares linear regression model to the prepared training data of the cars
from sklearn.linear_model import LinearRegression

linear_regression = LinearRegression()
linear_regression.fit(cars_prepared, cars_labels)

In [ ]:
# Generate some predictions
data = cars.iloc[:5]
pd.DataFrame(data)

In [ ]:
# Process the labels for the predictions
labels = cars_labels.iloc[:5]
pd.DataFrame(labels)

In [ ]:
# Use transform() instead of fit_transform to transform the test data, as not to transform the test data based on its properties instead of the full training set
data_prepared = full_pipeline.transform(data)
pd.DataFrame(data_prepared)

In [ ]:
# Time to make some predictions
predictions = linear_regression.predict(data_prepared).round()
predictions

In [ ]:
# Get the labels for comparison
labels

As we can see, the first test row predicts that the price is 22,768 when the actual value is 25,990. We move on to calculating the RMSE of predictions for the entirety of the training set, even though it is not really good practice to do testing on the trainingset. However, we will ignore this for now.

In [ ]:
from sklearn.metrics import mean_squared_error

linear_regression_cars_predictions = linear_regression.predict(cars_prepared)
linear_regression_mse = mean_squared_error(cars_labels, linear_regression_cars_predictions)
linear_regression_rmse = np.sqrt(linear_regression_mse)
np.round(linear_regression_rmse)

Looking at the results, this does not look very promising, as the typical error for the calculations is relatively high, compared to the price of the cars. Now, let us see if we are underfitting or overfittig the data, as this might be the cause for our problems.

In [ ]:
np.round(linear_regression_rmse / cars_labels.median(), 2)

Based on the results, it becomes evident that we are underfitting the data. This could potentially be becuase the linear regression model is too simple for the dataset. We therefore move on, and try a different model. Let us see how the Decision Tree Model performs.

In [ ]:
# Import and set up the model
from sklearn.tree import DecisionTreeRegressor

tree_regressor = DecisionTreeRegressor(random_state=42)
tree_regressor.fit(cars_prepared, cars_labels)

In [ ]:
# Add the RMSE
tree_regressor_cars_predictions = tree_regressor.predict(cars_prepared)
tree_regressor_mse = mean_squared_error(cars_labels, tree_regressor_cars_predictions)
tree_regressor_rmse = np.sqrt(tree_regressor_mse)
tree_regressor_rmse

Looks like we are overfitting the data. However, as the testing and creating of this code in Jupyter Notebook is done on a dataset that is way too small in a machine learning context, and that the "smaller files extracted from the larger ones are not random samples." As such, we can be fairly certain this is inpacting the accuracy of our results when working with the small datasets. Now, let us still try with Cross Validation.

In [ ]:
# Setting up and running the Cross Validation model for the used cars data
from sklearn.model_selection import cross_val_score

K = 10

tree_regressor_scores = cross_val_score(tree_regressor, cars_prepared, cars_labels,
                         scoring="neg_mean_squared_error", cv=K)
tree_regressor_rmse_scores = np.sqrt(-tree_regressor_scores)

In [ ]:
# The K scores can be summarised through the creation of a function
def display_scores(scores):
    print("Scores:", np.round(scores))
    print("Mean:", np.round(scores.mean()))
    print("Standard deviation:", np.round(scores.std()))

display_scores(tree_regressor_rmse_scores)

In [ ]:
# Compare if the performance to the results of linear regression
linear_regression_scores = cross_val_score(linear_regression, cars_prepared, cars_labels,
                                           scoring="neg_mean_squared_error", cv=K)
linear_regression_rmse_scores = np.sqrt(-linear_regression_scores)
display_scores(linear_regression_rmse_scores)

Cross Validation performs somewhat better than Linear Regression, even though none of them are yielding particularly qualitative results. Again, the factors of the size of the dataset, combined with how the data does not represent random samples, are fairly likely impacting the performance of our models. Let us still try with the Random Forest Model.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
forest_regressor.fit(cars_prepared, cars_labels)

forest_regressor_housing_predictions = forest_regressor.predict(cars_prepared)
forest_regressor_mse = mean_squared_error(cars_labels, forest_regressor_housing_predictions)
forest_regressor_rmse = np.sqrt(forest_regressor_mse)
forest_regressor_rmse.round()

In [ ]:
# Run this model through the cross validation model as well
forest_regressor_scores = cross_val_score(forest_regressor, cars_prepared, cars_labels,
                                          scoring="neg_mean_squared_error", cv=K)
forest_regressor_rmse_scores = np.sqrt(-forest_regressor_scores)
display_scores(forest_regressor_rmse_scores)

As we can see, the Decision Tree Model looks to be the best model out of the ones we tried. The standard deviation for the Decision tree Model is less than what it was for Linear Regression and the Random Forest Model. However, we still note that the data still is likely to be overfit in terms of the entire training set, considering how the score was substantially better on that one. As the differences are so small, because of insufficient data, we still see how the Random Forest Model performs on the test data.

### Improving and Fine-Tuning the Models

It has been decided that we will move forward with the Random Forest Model, based on the results. We now want to experiment with the hyperparameters of the Random Forest Model. As such, we must decide on what hyperparameters we want to experiment with, as well as what values we want to try.

In [ ]:
# Importing and setting up the GridSearchCV function
from sklearn.model_selection import GridSearchCV

parameter_grid = [
    # Try 12 (3×4) combinations of hyperparameters:
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},

    # Then try 6 (2×3) combinations with bootstrap set as False:
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]

forest_regressor = RandomForestRegressor(random_state=42)


grid_search = GridSearchCV(forest_regressor, parameter_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

grid_search.fit(cars_prepared, cars_labels)

In [ ]:
# Find the best hyperparameter values from the exploration so far
grid_search.best_estimator_

In [ ]:
# Check the RMSE, to establish all trialled combinations possible
grid_search_results = grid_search.cv_results_
for mean_score, params in zip(grid_search_results["mean_test_score"], grid_search_results["params"]):
    print(np.round(np.sqrt(-mean_score)), params)

In [ ]:
# Set the final model chosen to be the best out of the ones tried out
final_model = grid_search.best_estimator_

### Running the Test Set

Finally, it is time to try out the model on the test set. This has been set aside since we created it, as not to use or accidentally look at any of the data it contains, but now it is time to see how our chosen model performs on the test data.

In [ ]:
# Running the model on the test data for the dataset
X_test = stratified_test_set.drop("price", axis=1)
y_test = stratified_test_set["price"].copy()

X_test_prepared = full_pipeline.transform(X_test)

final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse.round()

In [ ]:
# Compare it to the mean
np.round(final_rmse / y_test.mean(), 2)

In [ ]:
# As the single RMSE value alone does not give any good indication for error variance, we try calculating the 95% confidence interval
from scipy import stats

confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
np.round(np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
                                        loc=squared_errors.mean(),
                                        scale=stats.sem(squared_errors))))


### Next Steps

Had this code been create for something other than an academic assessment setting, now would be the time to launch, monitor and maintain the system created. In this particular context, this is naturally not something we will do. However, we were able to explore the data structure, split it into a training set and a test set, clean the dataset, train a few models and find the best model from a few alternatives, when comparing the results.